# Gearing in swaps

In [2]:
import QuantLib as ql

Create a relinkable yield term structure handle and build a curve

In [1]:
import QuantLib as ql
import pandas as pd

yts = ql.YieldTermStructureHandle(ql.FlatForward(2, ql.TARGET(), 0.05, ql.Actual360()))
engine = ql.DiscountingSwapEngine(yts)
index = ql.USDLibor(ql.Period('6M'), yts)

schedule = ql.MakeSchedule(ql.Date(15,6,2021), ql.Date(15,6,2023), ql.Period('6M'))
nominal = [10e6]


fixedLeg = ql.FixedRateLeg(schedule, index.dayCounter(), nominal, [0.05])
floatingLeg = ql.IborLeg(nominal, schedule, index)
swap = ql.Swap(fixedLeg, floatingLeg)
swap.setPricingEngine(engine)

print(f"Floating leg NPV: {swap.legNPV(1):,.2f}\n")
pd.DataFrame([{
    'fixingDate': cf.fixingDate().ISO(),
    'accrualStart': cf.accrualStartDate().ISO(),
    'accrualEnd': cf.accrualEndDate().ISO(),
    "paymentDate": cf.date().ISO(),
    'gearing': cf.gearing(),
    'forward': cf.indexFixing(),
    'rate': cf.rate(),
    "amount": cf.amount()
} for cf in map(ql.as_floating_rate_coupon, swap.leg(1))])

Floating leg NPV: 933,741.01



,fixingDate,accrualStart,accrualEnd,paymentDate,gearing,forward,rate,amount
0,2021-06-11,2021-06-15,2021-12-15,2021-12-15,1.0,0.050641,0.050641,257424.241734
1,2021-12-13,2021-12-15,2022-06-15,2022-06-15,1.0,0.050637,0.050637,255999.698407
2,2022-06-13,2022-06-15,2022-12-15,2022-12-15,1.0,0.050641,0.050641,257424.241734
3,2022-12-13,2022-12-15,2023-06-15,2023-06-15,1.0,0.050637,0.050637,255999.698407


In [2]:
floatingLeg = ql.IborLeg(nominal, schedule, index, gearings=[0.7])
swap = ql.Swap(fixedLeg, floatingLeg)
swap.setPricingEngine(engine)

print(f"Floating leg NPV: {swap.legNPV(1):,.2f}\n")
pd.DataFrame([{
    'fixingDate': cf.fixingDate().ISO(),
    'accrualStart': cf.accrualStartDate().ISO(),
    'accrualEnd': cf.accrualEndDate().ISO(),
    "paymentDate": cf.date().ISO(),
    'gearing': cf.gearing(),
    'forward': cf.indexFixing(),
    'rate': cf.rate(),
    "amount": cf.amount()
} for cf in map(ql.as_floating_rate_coupon, swap.leg(1))])

Floating leg NPV: 653,618.71



,fixingDate,accrualStart,accrualEnd,paymentDate,gearing,forward,rate,amount
0,2021-06-11,2021-06-15,2021-12-15,2021-12-15,0.7,0.050641,0.035449,180196.969214
1,2021-12-13,2021-12-15,2022-06-15,2022-06-15,0.7,0.050637,0.035446,179199.788885
2,2022-06-13,2022-06-15,2022-12-15,2022-12-15,0.7,0.050641,0.035449,180196.969214
3,2022-12-13,2022-12-15,2023-06-15,2023-06-15,0.7,0.050637,0.035446,179199.788885


In [4]:
swapType = ql.VanillaSwap.Payer
numDates = (len(schedule)-1)
gearing = [0.7] * numDates
spread = [0.0] * numDates
fixedRateArray = [0.05] * numDates
nominalArray = nominal * numDates
nsSwap = ql.NonstandardSwap(
    swapType, nominalArray, nominalArray,
    schedule, fixedRateArray, index.dayCounter(),
    schedule, index, gearing, spread, index.dayCounter())

nsSwap.setPricingEngine(engine)
print(f"Floating leg NPV: {nsSwap.legNPV(1):,.2f}\n")
pd.DataFrame([{
    'fixingDate': cf.fixingDate().ISO(),
    'accrualStart': cf.accrualStartDate().ISO(),
    'accrualEnd': cf.accrualEndDate().ISO(),
    "paymentDate": cf.date().ISO(),
    'gearing': cf.gearing(),
    'forward': cf.indexFixing(),
    'rate': cf.rate(),
    "amount": cf.amount()
} for cf in map(ql.as_floating_rate_coupon, swap.leg(1))])

Floating leg NPV: 653,618.71



,fixingDate,accrualStart,accrualEnd,paymentDate,gearing,forward,rate,amount
0,2021-06-11,2021-06-15,2021-12-15,2021-12-15,1.0,0.050641,0.050641,180196.969214
1,2021-12-13,2021-12-15,2022-06-15,2022-06-15,1.0,0.050637,0.050637,179199.788885
2,2022-06-13,2022-06-15,2022-12-15,2022-12-15,1.0,0.050641,0.050641,180196.969214
3,2022-12-13,2022-12-15,2023-06-15,2023-06-15,1.0,0.050637,0.050637,179199.788885
